<a href="https://colab.research.google.com/github/maraes/Microtools/blob/main/C%C3%B3pia_de_9724243669bc7bc2cf2c1ba326963916.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mass auto transcriber

Creates transcription and caption of a media collection, whether video or audio.

Activate the GPU by following the steps:

- In top menu: `Runtime > Change runtime type > Hardware accelerator: GPU`

**Free Usage Limits** (approximately)
- With GPU, transcript around 30 hours of media per week.\
- With CPU, 10 hours per day

**Transcription Speed**

- GPU, 10 minutes of video in 1 minute. (10x) \
- CPU, 10 minute of video in 10 minutes. (1x) 

In [ ]:
import os
import time

from google.colab import drive, files
from pathlib import Path
from subprocess import run
from tqdm.notebook import tqdm

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install stable-ts
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-26oixlz_
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-26oixlz_
     |████████████████████████████████| 5.5 MB 26.7 MB/s 
     |████████████████████████████████| 182 kB 69.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.5 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175239 sha256=a1a40ad9c09ae0edb9a5997130ba706c2b97239afe2e945a00bfdfccd508c3f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-bwemliq9/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stable-ts: filename=stable_ts-1.0.0-py3-none-any.whl size=24728 sha256=1f61a73fc70ce019584ae89f8dc2767542118460a68d5238099a4edf

In [ ]:
#@title Import libs
import stable_whisper
import whisper
import torch

In [ ]:
# Connecting to a google account
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
colab_folder = r'/content/gdrive/MyDrive/Colab Notebooks/'
massautotranscriber_folder = Path(colab_folder) / 'mass_auto_transcribe/'
path_folder_input = Path(massautotranscriber_folder) / 'input'
path_folder_output = Path(massautotranscriber_folder) / 'output'

In [ ]:
#@title Set folders and Createif not exists
for folder_path in [colab_folder, massautotranscriber_folder, path_folder_input, path_folder_output]:
  if not Path(folder_path).exists():    
    Path(folder_path).mkdir()

Follow the steps:
- Access your [google drive](https://drive.google.com/)
- Access the folder `Colab Notebooks/mass_auto_transcriber/input`
- Upload the media files (video or audios) there

In [ ]:
#@title Download transcription model
print(torch.cuda.is_available())
model_type = 'small'
model = whisper.load_model(model_type)
model.device

True


device(type='cuda', index=0)

In [ ]:
#@title Make transcription

# Get media list to be transcript

list_audiopath = [x for x in Path(path_folder_input).iterdir()]
print('Existing media:' , ', '.join([x.stem for x in list_audiopath]))

list_file_transcript = [x for x in Path(path_folder_output).iterdir()]
print('Already transcripted:', ', '.join([x.stem for x in list_file_transcript]))

list_newaudiopath = [x for x in list_audiopath if x.stem not in [y.stem for y in list_file_transcript]]
print('To be transcripted:', ', '.join([x.stem for x in list_newaudiopath]))

Existing media: Arthur Schopenhauer - A Arte de ter Razão-O6Q23UiOKCk
Already transcripted: 
To be transcripted: Arthur Schopenhauer - A Arte de ter Razão-O6Q23UiOKCk


In [ ]:
for audio_path in tqdm(list_newaudiopath):
  start = time.time()
  result = model.transcribe(str(audio_path))
  end = time.time()
  filename = Path(audio_path).stem
  print(int(end - start), filename, '\n')
  # save transcription
  path_file_transcricao = Path(path_folder_output) / (filename + '.txt')
  with open(path_file_transcricao, "w") as f:
    f.write(result['text'])
  # save caption
  subtitle_path_output = Path(path_folder_output) / (filename + '.srt')
  stable_whisper.results_to_sentence_srt(result, str(subtitle_path_output))

  0%|          | 0/1 [00:00<?, ?it/s]

485 Arthur Schopenhauer - A Arte de ter Razão-O6Q23UiOKCk 

Saved: /content/gdrive/MyDrive/Colab Notebooks/mass_auto_transcribe/output/Arthur Schopenhauer - A Arte de ter Razão-O6Q23UiOKCk.srt


In [ ]:
#@title Generate a zip package of transcriptions
os.system(f'zip -r transcript.zip "{path_folder_output}"')

0

If you wish, in your google drive, delete the input and output content

In [ ]:
#@title Download zip file
files.download("transcript.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Delete online zip file
os.system(f'rm transcript.zip')